In [ ]:
#pip install arch

In [ ]:
# ==========================================================
# Maestría en Ciencia y Análisis de Datos
# Universidad Mayor de San Andrés
# ----------------------------------------------------------
#   Modelos lineales y modelos lineales generalizados
# ----------------------------------------------------------
#        Rolando Gonzales Martinez, Julio 2024
# ==========================================================
# Modelo lineal multivariante: (G)ARCH del TC blue bs/USD

import matplotlib.pyplot as plt
from arch import arch_model
from statsmodels.tsa.arima.model import ARIMA
import pandas as pd
import numpy as np

# Importar el conjunto de datos desde GitHub
url = 'https://raw.githubusercontent.com/rogon666/UMSA/main/MLMLG/datos/tc_bsusd_blue.csv'

# Cargar los datos en un DataFrame
data = pd.read_csv(url)
print(data.head())

In [ ]:
# Grafico de la serie de tiempo:
data['fecha'] = pd.to_datetime(data['fecha'])
plt.figure(figsize=(9, 4))
plt.plot(data['fecha'], data['tcblue'], label='tcblue', color='blue')
highlight_start = pd.Timestamp('2024-06-26')
plt.axvspan(highlight_start, data['fecha'].max(), color='gray', alpha=0.3)
plt.xlabel('Fecha')
plt.ylabel('tipo de cambio bs/USD')
plt.title('Datos diarios del tipo de cambio paralelo Bs/USD')
plt.legend()
plt.xlim(data['fecha'].min(), data['fecha'].max())
plt.text(highlight_start, 8, 'intento de golpe', color='gray', fontsize=9, rotation = 90)
plt.show()

In [ ]:
data.set_index('fecha', inplace=True)
data = data.asfreq('D')
# Modelo ARIMA
modelo = ARIMA(data['tcblue'], order=(5, 1, 0))
ajuste_modelo = modelo.fit()
resultados = ajuste_modelo.summary()
resultados

In [ ]:
# Grafico de ajuste del modelo ARIMA
data['residuos'] = ajuste_modelo.resid
std_residuals = data['residuos'].std()
outliers = data[abs(data['residuos']) > 3 * std_residuals]
data_cleaned = data.drop(outliers.index)
plt.figure(figsize=(4, 4))
plt.scatter(data_cleaned['tcblue'], ajuste_modelo.fittedvalues.loc[data_cleaned.index], color='blue', alpha=0.5)
plt.plot([data_cleaned['tcblue'].min(), data_cleaned['tcblue'].max()], [data_cleaned['tcblue'].min(), data_cleaned['tcblue'].max()], color='red', linestyle='--')
plt.xlabel('TC blue observado')
plt.ylabel('TC blue ajustado')
plt.title('TC blue observado y ajustado con modelo ARIMA')
plt.show()

In [ ]:
# Correlación serial (autocorrelación): Test de Breusch-Godfrey
from statsmodels.stats.diagnostic import acorr_breusch_godfrey
# nlags: retardos a testear por autocorrelación
bg_test = acorr_breusch_godfrey(ajuste_modelo, nlags=2)

# Resultados del test de Breusch-Godfrey
print(f'Estadístico LM: {bg_test[0]}')
print(f'Valor p (LM): {bg_test[1]}')
print(f'Estadístico F: {bg_test[2]}')
print(f'Valor p (F): {bg_test[3]}')

# Interpretación de los resultados
alpha = 0.05
if bg_test[1] < alpha:
    print("Se rechaza la hipótesis nula de no autocorrelación. \nExiste evidencia de autocorrelación.")
else:
    print("No se rechaza la hipótesis nula de no autocorrelación. \nNo hay evidencia de autocorrelación.")

In [ ]:
# Correlación serial (autocorrelación): test de Ljung-Box
from statsmodels.stats.diagnostic import acorr_ljungbox
# lags: todos los 1 a lag retardos a testear por autocorrelación
test_ljung_box = acorr_ljungbox(ajuste_modelo.resid, lags=10, return_df=True)
print(test_ljung_box)

In [ ]:
# Test ARCH de heteroscedasticidad (Engle)
import statsmodels.stats.diagnostic as diag
arch_test = diag.het_arch(ajuste_modelo.resid, nlags=4)
# Mostrar resultados del test
print(f"Estadigrafo LM Statistic: {arch_test[0]}")
print(f"p-value LM: {arch_test[1]}")
print(f"Estadigrafo F: {arch_test[2]}")
print(f"p-value F: {arch_test[3]}")

In [ ]:
# Modelo GARCH basado en los residuos ARIMA
modelo_garch = arch_model(data['residuos'].dropna(), vol='Garch', p=1, q=1)
garch_fit = modelo_garch.fit()
resultados_garch = garch_fit.summary()
print(resultados_garch)

In [ ]:
# Volatilidad
volatilidad_estimada = garch_fit.conditional_volatility
# Grafico de la volatilidad condicional estimada
start_date = pd.Timestamp('2024-04-01')
plt.figure(figsize=(10, 4))
plt.plot(data.loc[start_date:].index, volatilidad_estimada[data.index >= start_date], label='Volatilidad estimada (GARCH)', color='purple')
highlight_start = pd.Timestamp('2024-06-26')
plt.axvspan(highlight_start, data.index.max(), color='gray', alpha=0.3)
plt.xlabel('Fecha')
plt.ylabel('Volatilidad condicional')
plt.title('Volatilidad Estimada con un modelo GARCH para el tipo de cambio paralelo Bs/USD')
plt.legend()
plt.xlim(start_date, data.index.max())
plt.text(highlight_start, 0.03, ' periodo post-golpe', color='gray', fontsize=9)
plt.show()